In [1]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from bs4 import BeautifulSoup

# The next cell scrapes al the video ID's from an input file (html)

In [2]:
def scrape_video_ids(source_file_path):
    with open(source_file_path) as source:
        soup = BeautifulSoup(source, 'html.parser')
    extracted_id_list = []
    for link in soup.find_all("a"):
        try:
            href = link.get('href')
            if href.startswith('https://www.youtube.com'):
                # print(href)
                extracted_id_list.append(href.split('?v=')[1])
        except:
            pass
    return extracted_id_list

# The Next cell contains all the functions to authenticate with youtube and interact with the playlist

In [3]:
scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

def login():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "client_secret_TUIYouTube.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)
    return youtube


def create_playlist(youtube, playlist_title):
  request = youtube.playlists().insert(
      part="snippet,status",
      body={
        "snippet": {
          "title": playlist_title,
          "defaultLanguage": "en"
        },
        "status": {
          "privacyStatus": "public"
        }
      }
  )
  response = request.execute()

  print("Playlist Created: "+response['snippet']['title'])
  return response

def add_list_items(youtube, video_list):
  for video_id in video_list:
    request = youtube.playlistItems().insert(
      part="snippet",
      body={
        "snippet": {
          "playlistId": json_data['id'],
          "resourceId": {
            "kind": "youtube#video",
            "videoId": video_id
          }
        }
      }
    )
    response = request.execute()

    print("Video Added: "+response['snippet']['title'])

In [4]:
source_file = 'tui/CSC330-Reference/CSC330/Modules/Module1/Mod1Background.html'
video_id_list = scrape_video_ids(source_file)

login_data = login()
json_data = create_playlist(login_data, 'CSC330 Mod 1')
add_list_items(login_data, video_id_list)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1013734662046-24ejn6mn9pjepub3mvl3r9kurtuvqdvk.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=OXuEwDnmknUmfCYl4CcF1eGF0kGs1r&prompt=consent&access_type=offline
Playlist Created: CSC330 Mod 1
Video Added: Computer Parts Explained - Part 1:  Processors
Video Added: Computer Parts Explained - Part 2:  Motherboards
Video Added: Computer Parts Explained - Part 3: Graphics Cards
Video Added: Computer Parts Explained - Part 4: RAM Memory
Video Added: Computer Parts Explained - Part 5: Hard Drives
Video Added: Computer Parts Explained - Part 6: Computer Cases
Video Added: Computer Parts Explained - Part 7: PC Cooling
Video Added: Computer Parts Explained - Part 8: Power Supplies
Video Added: Computer Parts Explained - Part 9: Sound Cards
Video Added: Computer Parts Explain